# CoastVision Example: Waikiki
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 40px;">
        <p>CoastVision is an open-source Python framework geared towards generating satellite-derived shorelines (SDS) in <a href='https://developers.planet.com/docs/data/planetscope/'>PlanetScope</a> imagery. The image the right is two CoastVision SDS superimposed over drone imagery from the same week. This notebook provides an end-to-end example for Waikiki Beach Hawaiʻi.</p>
        <ol>
            <li>Download PlanetScope imagery for a given area of interest (AOI) and timeframe</li>
            <li>Image co-registration</li>
            <li>Shoreline extraction</li>
            <li>Tidal Correction</li>
            <li>QAQC</li>
        </ol>
    </div>
    <div>
        <img src="media\coastvision_shoreline_on_drone_hanaumapng.png" alt="SDS superimposed on high-resolution drone imagery" style="max-width: 100%; height: 100%;">
    </div>
</div>


# Import Libraries
If you have not yet set up enviromment in conda using `coastvision.yml`
```
cd path/to/CoastVision
conda env create -f coastvision.yml
conda activate coastvision
```

In [1]:
from coastvision import coastvisionRun
from planetscopeAPI import PlanetScopeOrdersAPI
from coastvision import setup
from coastvision import coastvisionCoreg

# 1. Download PlanetScope Imagery
<a href='https://developers.planet.com/docs/data/planetscope/'>PlanetScope</a> is a satellite constilation opperated by <a href='https://www.planet.com/'>Planet Labs Inc.</a> The PlanetScope constellatuion is made up of roughtly 130 satellites, capable of imageing the entire land surface of earth with daily revisit times and 3 meter spatial resolition. The imagery has four bands red, green, blue, and near-infrared.

In the following cell information regarding your site (beach or streatch of coastline) should be entered which will be used to download applicable satellite imagery from Planet
1. First enter (lat, long) coordinates (`coords`) creating an AOI (this can be any polygon) around a beach or coastline streatch you are interested in.
2. Next, create the `sitename` and `region` for the site
3. Enter a start and end date. Imagery from between and during these dates will be downloaded for the given API. Date format `YYYY-MM-DD`
4. Downloading PlanetScope imagery requires and API key. If you do not have a Planet account here is how you can create one: <a href='https://www.planet.com/get-started/'>Get Started with Planet</a>

To access your API key log into <a href='https://www.planet.com/'>Planet</a> and navigaet to "My Settings" (see image below).

<img src="media\api_key_planet.JPG" alt="API key in settings" style="max-width:70%">



In [6]:
# create AOI coords (this can be a poly gone with any number of points)
coords = [
    [-157.73757302038916,21.4064967434262],
    [-157.74027668707618,21.404379123418543],
    [-157.73611389868506,21.40058331266353],
    [-157.72770249121436,21.397626507579627],
    [-157.72611462347754,21.39910491759609],
    [-157.7328952478672,21.40226147273139],
    [-157.73757302038916,21.4064967434262]
    ]

region = 'kailua'
sitename = 'kailua'
setup.create_site_dict_json_for_API(
    site_name=sitename,
    aoi=coords,
    start_date="2024-01-01",
    end_date="2024-01-15")

setup.write_api_key_file(api_key='Your PlanetScope API Key') # this creates a text file that contains your API key and is referenced by PlaneScopeOrdersAPI

In [4]:
import importlib
importlib.reload(PlanetScopeOrdersAPI)
API = PlanetScopeOrdersAPI.PlanetScopeAPIOrder(selectSites=False, printPolling=True) # initalizing the class variable
API.get_all_data()

site name: kailua
{'name': 'kailua', 'products': [{'item_ids': ['20240114_210352_09_248f', '20240111_211433_19_24e1', '20240111_201953_51_24cc', '20240107_201508_73_2423', '20240107_201510_71_2423', '20240107_201742_32_24c8', '20240107_201744_53_24c8', '20240106_210139_54_247c', '20240106_210137_55_247c', '20240102_201846_68_24cc', '20240102_201848_91_24cc'], 'item_type': 'PSScene', 'product_bundle': 'analytic_udm2'}], 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[-157.73757302038916, 21.4064967434262], [-157.74027668707618, 21.404379123418543], [-157.73611389868506, 21.40058331266353], [-157.72770249121436, 21.397626507579627], [-157.72611462347754, 21.39910491759609], [-157.7328952478672, 21.40226147273139], [-157.73757302038916, 21.4064967434262]]]}}}, {'toar': {'scale_factor': 10000}}]}
is anything making it this far?
<Response [202]>
https://api.planet.com/compute/ops/orders/v2/5e05dbf6-1f59-4bdc-9394-c192f6597977
https://api.planet.com/compute/ops/orders/v2/5e0

# 2. Create Transects and Reference Shoreline
## 2.1 Transects
Coastal change is often measured through a series of shore normal transects. The intersection of SDS are computed and used to measure erosion and accreation along these transects. In this case the intersections are saved in a dataframe of the following structure:

<body>
    <table border="1">
        <tr>
            <th>Timestamp</th>
            <th>Transect Label 1</th>
            <th>Transect Label 2</th>
            <th>...</th>
            <th>Transect label n</th>
            <!-- Add more headers as needed -->
        </tr>
        <tr>
            <td>2019-03-03 20:43:11</td>
            <td>Distance along transect (m)</td>
            <td>...</td>
            <td>...</td>
            <td>...</td>
            <!-- Add more data cells as needed -->
        </tr>
        <!-- Add more rows as needed -->
    </table>
</body>

The distance along transect is the distance between the start (landward end) of the transect and where the shoreline intersects it.

*Note:* if you are only interested in shoreline contours and not transect intersections provide `just_extract_shoreline=False` in the `coastvisionRun.CoastVisionRun()` class initialization in 'Initialize CoastVision "Run" Class' below

## 2.2 Reference Shoreline
For quality control a reference shoreline can be hand digitized. Then given `max_dist_from_sl_ref = max distance in meters` supplied to `coastvision.CoastVisionRun()` all extracted shoreline segments out of this range are discarded. 
*Note:* This is recommended but not neccesary.


These should be saved in `user_inputs/<region>/<sitename>` as `<sitename>_transects.geojson` and `<sitename>_shoreline.geojson`. See <a href='https://github.com/Climate-Resilience-Collaborative/CoastVision/blob/1f50de7c3a477e864f28c5a5ab7c70a8e7016088/How_to_create_transect_and_sl_ref_files.md'>`How_to_create_transect_and_sl_ref_files.md`</a> for how to create these files.


<hr>

# 3. Initialize CoastVision "Run" Class

In [89]:
coastvision_run_class = coastvisionRun.CoastVisionRun(region=region, sitename=sitename)
print(f'There are {len(coastvision_run_class)} tiff images for site: {sitename}')

There are 11 tiff images for site: kailua


# 4. Image Co-registration
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 40px;">
        <p>Satellite images need to be accurately registered, meaning that they must align correctly with one another and with real-world coordinates. <a href="https://pypi.org/project/arosics/">AROSICS</a> and open-source Python package is used to co-register images to reduce error cause image missalignments. In the image below the right pane shows reduced image offsets after AROSICS co-registration. </p>
    </div>
    <div>
        <img src="media/arosics_logo.png" alt="Tidal Effect Example" style="max-width: 100%; height: auto;">
    </div>
</div>

![Co-registration Example](media/co-registration.gif)

In [81]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
coastvisionCoreg.coreg_site(region, sitename, grid_res=50, start=0)

kailua n 0; 0.00 percent; time elapsed: 0:00:00; 20240102_201846_68_2
performing coregistration
Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(630573.0, 2366688.0, 632049.0, 2367666.0)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(630573.0, 2366688.0, 632049.0, 2367666.0)
Matching window position (X,Y): 631239.2602571596/2367140.2614260665
Target window size (256, 256) not possible due to too small overlap area or window position too close to an image edge. New matching window size: (54, 54).
correcting shifts
Initializing tie points grid...
Calculating tie point grid (20 points) using 8 CPU cores...
Found 15 matches.
Performing validity checks...
0 tie points flagged by level 1 filtering (reliability).
14 tie points flagged by level 2 filtering (SSIM).
1 valid tie points remain

# 5. Extract Shorelines and Compute Transect Intersections

The following function runs through downloaded satellite doing the following:

1. Segment image into land and water
2. Extract shoreline
3. Compute shoreline intersection with transects


<img src="media/stages_plot.jpg" alt="Stages Plot">


This function saves the `intersection_df` at the following path: `outputs/<region/<sitename>_transect_intersections.csv`

In [90]:
intersection_df = coastvision_run_class.run_shoreline_extraction()
display(intersection_df.head(2))

(326, 492)
 kailua 81.82 percent progress 20240111_211433_19_24e1skipping this image because it doesn't contain any of the sl ref: c:\Users\jcnic\OneDrive\Documents\college\research\SOEST\CoastVision\data\kailua\kailua\20240111_211433_19_24e1_3B_AnalyticMS_toar_clip.tif
 kailua 100.0 percent progress 20240102_201846_68_24cc

,0,1,2,3,4,5,6,7
2024-01-02 20:18:46.680,83.657963,78.955000,71.131828,67.858131,72.228374,55.106166,61.179543,71.712721
2024-01-02 20:18:48.910,82.662877,78.689398,69.013657,65.685224,70.469556,52.991976,61.345805,72.149708


# 6. Tidal Corrections


## 6.1 Tidal Effects on Horizontal Shoreline Position
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 20px;">
        <p>CoastVision shoreline accuracy can be improved by correcting for horizontal shifts in shoreline position due to tidal changes. The example to the right from Cadíz, Spain shows how large horizontal changes from tidal shifts can be. Using tide level data corrections can be made to where the shoreline position would be given mean sea level removing noise added by tidal fluctuations</p>
    </div>
    <div>
        <img src="media/tide_horizontal_shift_example.gif" alt="Tidal Effect Example" style="max-width: 100%; height: auto;">
    </div>
</div>


<img src="media\tidal_horizontal_effect_figure.png" alt="Tidal Effect Figure">


## 6.2 Tidal Corrections via Global Tide Model
The Finite Element Solution (FES) numerical model can be used to calculate ocean tides and their effect on sea level. These ocean tide height predictions can be used along with a beach slope estimate to compute the horizontal shoreline position correction. [CoastSat.slope](https://github.com/kvos/CoastSat.slope) provides [documentation](https://github.com/kvos/CoastSat.slope/blob/master/doc/FES2014_installation.md) on how to download and use this. 

## 6.3 Tidal Corrections via Local Tide Gauge Data
If local tide gauge data is avaible this can be used rather than FES predictions.


The tidal corrected dataframe is saved as here: `outputs/<region/<sitename>_intersections_tidally_corrected_<reference_elevation>m.csv`


In [ ]:
# NOTE: EXPLAIN WHAT REFERENCE ELEVATION IS
# tidal_corrected_df = coastvision_run_class.tidal_correction(reference_elevation=0)
# display(tidal_corrected_df)

# 7. QAQC
There are many sources of error in satellite-derived shorelines from faulty image classification to image alignment to tidal and wave runup noise. While much of this noise is cancelled out by the wealth of data (PlanetScope has near-daily revisit time) it is still important to remove outliers. Below median filtering is used to remove outliers that are not within the `limit` (in meters) of the median (`median - limit < x < median + limit`). This filtered transect intersection dataframe is saved `outputs/<region/<sitename>_QAQC_transect_interesections.csv`.

In [77]:
QAQC_df = coastvision_run_class.intersection_QAQC(limit=10)
display(QAQC_df.head(2))

before QAQC: shape (11, 8) number of nans 11
after QAQC: shape (11, 8) number of nans 15


,0,1,2,3,4,5,6,7
2024-01-02 20:18:46.680,83.657963,78.955000,71.131828,67.858131,72.228374,55.106166,61.179543,71.712721
2024-01-02 20:18:48.910,81.543799,78.807638,69.058971,65.970027,70.056018,52.356425,60.130634,72.070998
